In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta

# ----- Data Generation Section -----
def random_date(start, end):
    """Return a random date between two datetime objects."""
    delta = end - start
    random_days = random.randint(0, delta.days)
    return start + timedelta(days=random_days)

def generate_item_id(n):
    """Generate item IDs with a fixed format."""
    return [f"ITEM{str(i).zfill(4)}" for i in range(1, n+1)]

def generate_container_id(n):
    """Generate container IDs with a fixed format."""
    return [f"CONT{str(i).zfill(3)}" for i in range(1, n+1)]

# Settings for synthetic data generation
NUM_ITEMS = 1000      # Number of items to generate
NUM_CONTAINERS = 50   # Number of containers to generate

# Define possible names for items and zones for containers/items
item_names = [
    "Food Packet", "Oxygen Cylinder", "First Aid Kit",
    "Tool Box", "Spare Part", "Scientific Instrument"
]
zones = ["Crew Quarters", "Airlock", "Laboratory"]

# Generate items DataFrame
item_ids = generate_item_id(NUM_ITEMS)
items_data = {
    "itemId": item_ids,
    "name": [random.choice(item_names) for _ in range(NUM_ITEMS)],
    # Dimensions in cm (with realistic ranges)
    "width": np.random.randint(5, 31, size=NUM_ITEMS),   # 5 to 30 cm
    "depth": np.random.randint(5, 31, size=NUM_ITEMS),
    "height": np.random.randint(5, 31, size=NUM_ITEMS),
    # Mass in kg (random between 1 and 20)
    "mass": np.random.uniform(1, 20, size=NUM_ITEMS).round(2),
    # Priority between 1 and 100
    "priority": np.random.randint(1, 101, size=NUM_ITEMS),
    # Expiry date: random date between tomorrow and 2 years from now
    "expiryDate": [
        random_date(datetime.now() + timedelta(days=1), datetime.now() + timedelta(days=730)).strftime("%Y-%m-%d")
        for _ in range(NUM_ITEMS)
    ],
    # Usage limit between 1 and 50
    "usageLimit": np.random.randint(1, 51, size=NUM_ITEMS),
    # Preferred zone from the zones list
    "preferredZone": [random.choice(zones) for _ in range(NUM_ITEMS)]
}
df_items = pd.DataFrame(items_data)
# Compute additional feature: volume (width * depth * height)
df_items["volume"] = df_items["width"] * df_items["depth"] * df_items["height"]

# Generate containers DataFrame
container_ids = generate_container_id(NUM_CONTAINERS)
containers_data = {
    "containerId": container_ids,
    "zone": [random.choice(zones) for _ in range(NUM_CONTAINERS)],
    # Dimensions in cm: larger values to simulate storage containers
    "width": np.random.randint(50, 201, size=NUM_CONTAINERS),   # 50 to 200 cm
    "depth": np.random.randint(50, 201, size=NUM_CONTAINERS),
    "height": np.random.randint(100, 301, size=NUM_CONTAINERS)    # 100 to 300 cm
}
df_containers = pd.DataFrame(containers_data)

# Save the datasets as CSV files (visible in Kaggle Files tab)
df_items.to_csv("dummy_items.csv", index=False)
df_containers.to_csv("dummy_containers.csv", index=False)

print("Dummy datasets generated successfully:")
print("Items shape:", df_items.shape)
print("Containers shape:", df_containers.shape)

# ----- Simulation Section -----
# Container class to simulate placement and retrieval
class Container:
    def __init__(self, container_id, zone, width, depth, height):
        self.container_id = container_id
        self.zone = zone
        self.width = width
        self.depth = depth
        self.height = height
        self.items = []  # List to hold items with placement info

    def available_volume(self):
        # Use item["volume"] if exists; otherwise, calculate volume from dimensions
        used_volume = sum(
            item["volume"] if "volume" in item else (item["width"] * item["depth"] * item["height"])
            for item in (entry["item"] for entry in self.items)
        )
        return self.width * self.depth * self.height - used_volume

    def add_item(self, item, position):
        """Place item with a simulated position index."""
        self.items.append({"item": item, "position": position})
    
    def remove_item(self, item_id):
        self.items = [entry for entry in self.items if entry["item"]["itemId"] != item_id]
    
    def get_retrieval_steps(self, item_id):
        """Simulate retrieval by counting the items in front of the requested item."""
        sorted_items = sorted(self.items, key=lambda x: x["position"])
        steps = 0
        for entry in sorted_items:
            if entry["item"]["itemId"] == item_id:
                break
            steps += 1
        return steps

# Functions to load datasets and initialize containers
def load_datasets(items_csv="dummy_items.csv", containers_csv="dummy_containers.csv"):
    df_items = pd.read_csv(items_csv)
    df_containers = pd.read_csv(containers_csv)
    return df_items, df_containers

def initialize_containers(df_containers):
    containers = {}
    for _, row in df_containers.iterrows():
        cont = Container(
            container_id=row["containerId"],
            zone=row["zone"],
            width=row["width"],
            depth=row["depth"],
            height=row["height"]
        )
        containers[row["containerId"]] = cont
    return containers

def placement_algorithm(df_items, containers):
    """
    Greedy placement:
      - Try to place each item in a container within its preferred zone if space allows.
      - If no container in the preferred zone fits, use any container with sufficient space.
      - Simulate the position using the current count of items in that container.
    """
    placements = []
    for _, item in df_items.iterrows():
        candidate = None
        # First, search containers in the preferred zone
        for container in containers.values():
            if container.zone == item["preferredZone"]:
                required_volume = item["width"] * item["depth"] * item["height"]
                if container.available_volume() >= required_volume:
                    if candidate is None or container.available_volume() > candidate.available_volume():
                        candidate = container
        # If not found, search all containers
        if candidate is None:
            for container in containers.values():
                required_volume = item["width"] * item["depth"] * item["height"]
                if container.available_volume() >= required_volume:
                    if candidate is None or container.available_volume() > candidate.available_volume():
                        candidate = container
        # Place the item if a candidate is found
        if candidate is not None:
            position = len(candidate.items)
            candidate.add_item(item, position)
            placements.append({
                "itemId": item["itemId"],
                "containerId": candidate.container_id,
                "position": position
            })
        else:
            placements.append({
                "itemId": item["itemId"],
                "containerId": None,
                "position": None,
                "error": "Insufficient space"
            })
    return placements

def simulate_retrieval(containers, item_id):
    """
    Simulate retrieval by identifying the container holding the item
    and counting the number of items before it.
    """
    for container in containers.values():
        for entry in container.items:
            if entry["item"]["itemId"] == item_id:
                steps = container.get_retrieval_steps(item_id)
                return {
                    "containerId": container.container_id,
                    "retrievalSteps": steps
                }
    return {"containerId": None, "retrievalSteps": None, "error": "Item not found"}

def simulate_waste_management(containers, current_date):
    """
    Identify items that become waste due to expiry or depleted usage.
    For simplicity, each retrieval reduces the usage limit by 1.
    """
    waste_items = []
    for container in containers.values():
        for entry in container.items.copy():
            item = entry["item"]
            expiry = datetime.strptime(item["expiryDate"], "%Y-%m-%d")
            if expiry < current_date or item["usageLimit"] <= 0:
                waste_items.append({
                    "itemId": item["itemId"],
                    "name": item["name"],
                    "containerId": container.container_id,
                    "reason": "Expired" if expiry < current_date else "Out of Uses"
                })
                container.remove_item(item["itemId"])
    return waste_items

def simulate_day(containers, retrieval_requests, current_date):
    """
    Simulate one day of operations:
      - Process retrieval requests.
      - Decrement usage limits for retrieved items.
      - Run waste management at the end of the day.
    """
    retrieval_results = {}
    for item_id in retrieval_requests:
        result = simulate_retrieval(containers, item_id)
        if result.get("containerId"):
            # Decrement usageLimit from the item
            for container in containers.values():
                for entry in container.items:
                    if entry["item"]["itemId"] == item_id:
                        entry["item"]["usageLimit"] = max(0, entry["item"]["usageLimit"] - 1)
                        retrieval_results[item_id] = result
                        break
    waste = simulate_waste_management(containers, current_date)
    return retrieval_results, waste

def main_simulation():
    # Load datasets (generated earlier)
    df_items, df_containers = load_datasets()
    
    # Initialize container objects
    containers = initialize_containers(df_containers)
    
    # Perform placement algorithm (simulate resupply mission)
    placements = placement_algorithm(df_items, containers)
    print("\n=== Placement Recommendations (Sample) ===")
    for p in placements[:5]:
        print(p)
    
    # Simulation parameters
    current_date = datetime.now()
    simulation_days = 5
    all_item_ids = df_items["itemId"].tolist()
    
    # Run simulation for a number of days
    for day in range(1, simulation_days + 1):
        print(f"\n=== Day {day} Simulation ===")
        retrieval_requests = random.sample(all_item_ids, 10)  # Randomly pick 10 items to retrieve
        retrieval_results, waste = simulate_day(containers, retrieval_requests, current_date)
        
        print("Retrieval Results (sample):")
        for item_id, res in retrieval_results.items():
            print(f"Item {item_id}: {res}")
        
        if waste:
            print("Waste Items Identified:")
            for waste_item in waste:
                print(waste_item)
        else:
            print("No waste items identified today.")
        
        # Move to the next day
        current_date += timedelta(days=1)
    
    print("\n=== Final Container States ===")
    for container in containers.values():
        print(f"{container.container_id} has {len(container.items)} items remaining.")

# Run the simulation
main_simulation()




Dummy datasets generated successfully:
Items shape: (1000, 11)
Containers shape: (50, 5)

=== Placement Recommendations (Sample) ===
{'itemId': 'ITEM0001', 'containerId': 'CONT019', 'position': 0}
{'itemId': 'ITEM0002', 'containerId': 'CONT019', 'position': 1}
{'itemId': 'ITEM0003', 'containerId': 'CONT021', 'position': 0}
{'itemId': 'ITEM0004', 'containerId': 'CONT021', 'position': 1}
{'itemId': 'ITEM0005', 'containerId': 'CONT019', 'position': 2}

=== Day 1 Simulation ===
Retrieval Results (sample):
Item ITEM0619: {'containerId': 'CONT019', 'retrievalSteps': 184}
Item ITEM0975: {'containerId': 'CONT011', 'retrievalSteps': 121}
Item ITEM0858: {'containerId': 'CONT021', 'retrievalSteps': 112}
Item ITEM0475: {'containerId': 'CONT019', 'retrievalSteps': 159}
Item ITEM0798: {'containerId': 'CONT019', 'retrievalSteps': 208}
Item ITEM0725: {'containerId': 'CONT002', 'retrievalSteps': 1}
Item ITEM0716: {'containerId': 'CONT011', 'retrievalSteps': 91}
Item ITEM0297: {'containerId': 'CONT019',